In [1]:
# Torch
import torch

# Custom imports
import sys
sys.path.append('./')
from NeuralNets import FNN, KAN
from system import LotkaVolterra
from upinn import UPINN

In [2]:
alpha, beta, gamma, delta = 2/3, 4/3, 1.0, 1.0
x0, y0 = 1.0, 1.0
LV = LotkaVolterra(alpha, beta, gamma, delta, torch.tensor([x0, y0], dtype=torch.float32))

In [3]:
# hidden = [16] * 4
# u = FNN(
#     dims=[1, *hidden, 2],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Softplus(),
# )
# G = FNN(
#     dims=[2, *hidden, 2],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.ReLU(),
# )

hidden = [4] * 4
u = KAN(
    layers_hidden=[1, *hidden, 2],
)

G = KAN(
    layers_hidden=[2, *hidden, 2],
)

# Setup scaling layer
u.scale_fn = lambda t_: (t_-LV.t.min())/(LV.t.max()-LV.t.min())
mu, sigma = 0, 2
epsilon = 1e-8
G.scale_fn = lambda x: (x-mu)/(sigma+epsilon)

# params = dict(alpha=2/3, beta=4/3, delta=1.0)
params = dict()

upinn = UPINN(u, G, LV, params)

upinn.train(
    optimizer=torch.optim.AdamW,
    optimizer_args=dict(lr=3e-3, weight_decay=3e-10),

    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
    scheduler_args=dict(factor=0.9, patience=50, min_lr=1e-6),
    scheduler_extend=torch.optim.lr_scheduler.ReduceLROnPlateau,
    scheduler_extend_args=dict(factor=0.9, patience=100, min_lr=1e-6),

    epochs=30000,
    loss_tol=5e-6,
    extend_domain=True,
    log_wandb=dict(log=True, name='LV-UPINN-FNN', project='Master-Thesis', log_plots=True, plot_interval=1000),
    beta_softadapt=0.1,
)

Beginning training...
Running on: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: megajosni. Use `wandb login --relogin` to force relogin
wandb: WARNING Path <TemporaryDirectory 'C:\\Users\\jonas\\AppData\\Local\\Temp\\tmpo3n2rllm'>\wandb\ wasn't writable, using system temp directory.


KeyboardInterrupt: 